Explaining MAB

- The MAB agent problem formulation involves finding the ad that maximizes the overall click-through rate (CTR) while balancing the trade-off between exploiting the ad that seems to be the most effective so far and exploring other ads to potentially find a better one. The agent must decide which ad to show at each time step, based on the observed data and the strategy used. The goal is to maximize the expected total reward over a fixed time horizon, taking into account the uncertainty of the rewards and the limited budget for testing each ad.
- The MAB agent must balance the exploration of less-known ads to learn their CTRs with the exploitation of the ads that are known to have higher CTRs to maximize the total number of clicks.

In [ ]:
import numpy as np
import pandas as pd
import random

In [ ]:
ads_clicks = pd.read_csv("/content/Ads_Optimisation.csv")

In [ ]:
ads_clicks

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,0,0,1,0,0,0,0,1,0,0
9996,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0
9998,1,0,0,0,0,0,0,1,0,0


In [ ]:
ads_clicks.describe

<bound method NDFrame.describe of       Ad 1  Ad 2  Ad 3  Ad 4  Ad 5  Ad 6  Ad 7  Ad 8  Ad 9  Ad 10
0        1     0     0     0     1     0     0     0     1      0
1        0     0     0     0     0     0     0     0     1      0
2        0     0     0     0     0     0     0     0     0      0
3        0     1     0     0     0     0     0     1     0      0
4        0     0     0     0     0     0     0     0     0      0
...    ...   ...   ...   ...   ...   ...   ...   ...   ...    ...
9995     0     0     1     0     0     0     0     1     0      0
9996     0     0     0     0     0     0     0     0     0      0
9997     0     0     0     0     0     0     0     0     0      0
9998     1     0     0     0     0     0     0     1     0      0
9999     0     1     0     0     0     0     0     0     0      0

[10000 rows x 10 columns]>

In [ ]:
num_ads = len(ads_clicks.columns)
num_steps = len(ads_clicks)

In [ ]:
# Initialize action-value estimates for each ad to 0
estimates = np.zeros(num_ads)

##### Defining the epsilon-greedy method

In [ ]:
# Define the ε-greedy action method
def epsilon_greedy(epsilon):
    if random.random() < epsilon:
        return random.randint(0, num_ads-1)
    else:
        return np.argmax(estimates)

##### Defining the ucb method

In [ ]:
# Define the UCB action method 
def ucb(c):
    # Compute the UCB value for each ad
    n = np.sum(action_counts)
    ucb_values = estimates + c * np.sqrt(np.log(n) / (action_counts + 1e-5))
    # Choose the ad with the highest UCB value
    return np.argmax(ucb_values)

Now, let's use these functions to compute the total rewards after 2000 time steps for both the ε-greedy and UCB action methods:

In [ ]:
# Initialize counts for each ad to 0
action_counts = np.zeros(num_ads)

##### ε-greedy action with ε=0.01 

In [ ]:
# Run the ε-greedy action with ε=0.01 for 2000 time steps
epsilon = 0.01
total_reward_eps01 = 0
for t in range(num_steps):
    action = epsilon_greedy(epsilon)
    action_counts[action] += 1
    reward = ads_clicks.iloc[t, action]
    try:
      total_reward_eps01 += reward
      estimates[action] += (reward - estimates[action]) / action_counts[action]
    except:
      continue

For ε = 0.01, the ε-greedy action will mostly exploit the ad that has the highest estimated value, and only occasionally explore other ads. This can lead to a slow convergence to the optimal ad, but once the agent has a good estimate of the best ad, it will consistently choose it and achieve high rewards. 

##### ε-greedy action with ε=0.3 

In [ ]:
from os import execlp
# Run the ε-greedy action with ε=0.3 for 2000 time steps
epsilon = 0.3
total_reward_eps03 = 0
for t in range(num_steps):
    action = epsilon_greedy(epsilon)
    action_counts[action] += 1
    reward = ads_clicks.iloc[t, action]
    #print(reward)
    #print("\n")
    try:
      total_reward_eps03 += reward
      estimates[action] += (reward - estimates[action]) / action_counts[action]
    except:
      continue

For ε = 0.3, the agent will explore more frequently, which can lead to a faster convergence to the optimal ad but can also result in lower rewards if the exploration is too random.

In [ ]:
# Run the UCB action with c=1.5 for 2000 time steps
c = 1.5
total_reward_ucb = 0
for t in range(num_steps):
    # Choose an ad using the UCB action
    action = ucb(c)
    # Update the count and estimated value for the chosen ad
    action_counts[action] += 1
    reward = ads_clicks.iloc[t, action]
    total_reward_ucb += reward
    estimates[action] += (reward - estimates[action]) / action_counts[action]

In [ ]:
print("Total reward for ε-greedy with ε=0.01:", total_reward_eps01)
print("Total reward for ε-greedy with ε=0.3:", total_reward_eps03)
print("Total reward for UCB with c=1.5:", total_reward_ucb)

Total reward for ε-greedy with ε=0.01: 2171
Total reward for ε-greedy with ε=0.3: 2274
Total reward for UCB with c=1.5: 2315


##### Explaining how the action value estimate compares to optimal action 

- For all approaches, the action value estimated will initially be uncertain, but as more data is collected, the estimate will become more accurate. 
- The ε-greedy action with ε = 0.01 will eventually converge to the optimal ad and achieve high rewards, but this can take longer than the more exploratory ε-greedy action with ε = 0.3 or the UCB action method. 
- The UCB action method will explore more systematically and converge faster to the optimal ad, but its performance can be sensitive to the choice of the parameter c. 
- Overall, the success of each approach will depend on the specific characteristics of the data and the environment.

- In the MAB problem, the goal is to maximize the total reward obtained over a sequence of actions. The optimal action at any time step is the one that would result in the highest expected reward, given the current state of the system.

- For the ε-greedy action method, the estimated action values are updated based on the rewards obtained for each action over time. The estimated value for each action is calculated as the average reward obtained when that action is taken. As the number of times each action is taken increases, the estimates become more accurate. However, the estimates may be biased if certain actions are not taken often enough, which can lead to suboptimal performance.

- The UCB action method addresses the bias problem by taking into account not only the average reward obtained for each action, but also the variance in the rewards and the number of times each action has been taken. The UCB action method chooses actions that have a high estimated value as well as high uncertainty, which can lead to more exploration of the action space and better overall performance.

- In general, both the ε-greedy and UCB action methods will converge to the optimal action over time as more data is collected and the estimates become more accurate. 
- However, the UCB action method may converge faster since it takes into account uncertainty and encourages exploration of less certain actions. 
- In some cases, the ε-greedy action method may get stuck in a suboptimal action for longer if it is not able to explore other options effectively.